In [ ]:
import prehook

prehook.execute_prehook()

In [ ]:
import hook
import prehook
import database_handler
import lookups
import datetime

db_session = database_handler.create_connection()
# hook.create_etl_checkpoint(db_session)

In [ ]:
hook.return_lookup_items_as_dict(lookups.IncrementalField)

In [1]:
import hook
import prehook
import database_handler
import lookups
import datetime

db_session = database_handler.create_connection()
etl_date = datetime.datetime(1900,1,1)
#SourceName is schema name
source_name = lookups.SourceName.DVD_RENTAL.value
#This will get you a list of tables inside this schema that are mentioned in SQLTablesToReplicate
tables = prehook.return_tables_by_schema(source_name)
#This will return 
incremental_date_dict = hook.return_lookup_items_as_dict(lookups.IncrementalField)
# for table in tables:
staging_query = f"""
        SELECT * FROM {source_name}.{tables[0]} WHERE {incremental_date_dict.get(tables[0])} >= '{etl_date}'
""" 
staging_df = database_handler.return_data_as_df(db_session= db_session, input_type= lookups.InputTypes.SQL, file_executor= staging_query)
dst_table = f"stg_{source_name}_{tables[0]}"
insert_stmt = database_handler.return_insert_into_sql_statement_from_df(staging_df, 'dw_reporting', dst_table)
for insert in insert_stmt:
        database_handler.execute_query(db_session=db_session, query= insert)

c:\Users\96171\Desktop\SE_Factory\Week7\ETL\ETL_Project_2\database_handler.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return_dataframe = pd.read_sql_query(con= db_session, sql= file_executor)
